# MACD

In [ ]:
from Risk.NullRiskManagementModel import NullRiskManagementModel
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Algorithm")
AddReference("QuantConnect.Indicators")
AddReference("QuantConnect.Common")

from System import *
from QuantConnect import *
from QuantConnect.Algorithm import *
from QuantConnect.Indicators import *
from datetime import datetime

In [ ]:
#for more data review:
#https://www.quantconnect.com/research/notebook/16213/MyResearch.ipynb
#usamos la paqueteria de señales MACD
class MACDTrendAlgorithm(QCAlgorithm):

    def Initialize(self):
        '''Initialise the data and resolution required, as well as the cash and start-end dates for your algorithm. All algorithms must initialized.'''
        self.ticker='NVDA'#add the ticker, just stocks
        #we can also trade fx or cryptos, but we need to chang add equity
        self.SetStartDate(2014, 1, 1)    #Set Start Date
        self.SetEndDate(DateTime.Today)      #Set End Date
        self.SetCash(1000)             #Set Strategy Cash
        # Find more symbols here: http://quantconnect.com/data
        self.AddEquity(self.ticker, Resolution.Hour)#daily,hour, minute, tick
        #self.AddForex(self.ticker,Resolution.Hour)
        #self.AddCrypto(self.ticker,Resolution.Minute)
        # define our daily macd(12,26) with a 9 hour signal
        # how to trade macd https://www.investopedia.com/terms/m/macd.asp
        self.__macd = self.EMA(self.ticker, 12, 26,9, MovingAverageType.Exponential,Resolution.Hour)
        self.__previous = datetime.min
        self.PlotIndicator("MACD", True, self._macd, self._macd.Signal)
        self.PlotIndicator(self.ticker, self._macd.Fast, self._macd.Slow)


    def OnData(self, data):
        '''OnData event is the primary entry point for your algorithm. Each new data point will be pumped in here.'''
        # wait for our macd to fully initialize
        if not self.__macd.IsReady: return#se inicializa el macd

        # only once per hour
        if self.__previous.date() == self.Time.date(): return#compara precio anterior con actual

        # define a small tolerance on our checks to avoid bouncing
        tolerance = 0.0025#tolerancia de reobote

        holdings = self.Portfolio[self.ticker].Quantity#Ccantidad de titulos

        signalDeltaPercent = (self._macd.Current.Value - self.macd.Signal.Current.Value)/self._macd.Fast.Current.Value
#se presenta la señal de cambio
        # if our macd is greater than our signal, then let's go long
        #señal para comprar si se tiene <= que cero titulos
        if holdings <= 0 and signalDeltaPercent > tolerance:  
            # longterm says buy as well
            self.SetHoldings(self.ticker, 1.0)
#set existing holdings
#self.SetHoldings(Symbol symbol, double percentage, bool liquidateExistingHoldings = false)

        # of our macd is less than our signal, then let's go short
        elif holdings >= 0 and signalDeltaPercent < -tolerance:
            self.Liquidate(self.ticker)#Se venden los titulos que teniamos
#liquidate holdings
#self.Liquidate(symbolToLiquidate=None) # set ticker to liquidate specific symbol
        self.__previous = self.Time
#for more than 1 ticker
# Set fixed percentages of known tickers:
# self.SetHoldings("IBM", 0.25)
# self.SetHoldings("GOOG", 0.25)
# self.SetHoldings("AAPL", 0.25)
# self.SetHoldings("MSFT", 0.25)

# # Or set portfolio to equal weighting of all securities:
# weighting = 1.0 / self.Securities.Count
# for security in self.Securities.Values:
#     self.SetHoldings(security.Symbol, weighting)